In [6]:
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [10]:
import pandas as pd
import mysql.connector
import os

# Function to create the database if it doesn't exist
def create_database(cursor, database_name):
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        print(f"Database '{database_name}' ensured.")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")

# Function to insert data from DataFrame to MySQL
def insert_data_from_df_to_mysql(df, connection):
    try:
        cursor = connection.cursor(dictionary=True)
        # Create table if it does not exist
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS bus_details (
            Route_Name TEXT,
            Route_Link TEXT,
            Bus_Name TEXT,
            Bus_Type TEXT,
            Departing_Time TIME,
            Duration TEXT,
            Reaching_Time TIME,
            Star_Rating FLOAT,
            Price DECIMAL(10,2),
            Seat_Availability VARCHAR(255)
        )
        """)

        # Insert data into the table
        for _, detail in df.iterrows():
            cursor.execute("""
            INSERT INTO bus_details (
                Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time,
                Duration, Reaching_Time, Star_Rating, Price, Seat_Availability
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                detail['Route_Name'], detail['Route_Link'], detail['Bus_Name'],
                detail['Bus_Type'], detail['Departing_Time'], detail['Duration'],
                detail['Reaching_Time'], detail['Star_Rating'], detail['Price'],
                detail['Seat_Availability']
            ))

        # Commit the transaction
        connection.commit()
        cursor.close()
        print("Data inserted successfully into MySQL database.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Main function to connect to MySQL, create database and insert data
def main():
    database_name = 'redbus_1'

    try:
        # Establish connection to MySQL server (without specifying database)
        connection = mysql.connector.connect(
            host='127.0.0.1',
            port='3306',
            user='root',
            password='1234'
        )

        cursor = connection.cursor()
        # Create the database if it doesn't exist
        create_database(cursor, database_name)
        cursor.close()
        connection.close()

        # Reconnect to the specified database
        connection = mysql.connector.connect(
            host='127.0.0.1',
            port='3306',
            user='root',
            password='1234',
            database=database_name
        )

        # List of CSV files to be inserted into the database
        csv_files = [
            r'C:\Users\Administrator\Desktop\redbus\jksrtc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\apsrtc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\ctutc_bus_deatails.csv',
            r'C:\Users\Administrator\Desktop\redbus\hrtc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\ksrtc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\ktcl_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\sbstc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\kaac_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\upsrtc_bus_details.csv',
            r'C:\Users\Administrator\Desktop\redbus\wbtc_bus_details.csv'
        ]

        # Loop through each CSV file and insert data into MySQL
        for csv_file in csv_files:
            if os.path.exists(csv_file):
                df = pd.read_csv(csv_file)
                insert_data_from_df_to_mysql(df, connection)
            else:
                print(f"File not found: {csv_file}")

        connection.close()

    except mysql.connector.Error as err:
        print(f"Error: {err}")

if __name__ == "__main__":
    main()


Database 'redbus_1' ensured.
Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.
Error: 1366 (HY000): Incorrect decimal value: 'INR 676' for column 'Price' at row 1
Error: 1366 (HY000): Incorrect decimal value: 'INR 507' for column 'Price' at row 1
Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.
Error: 1366 (HY000): Incorrect decimal value: 'INR 95' for column 'Price' at row 1
Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.


In [11]:
import pandas as pd
import mysql.connector
import os

# Function to preprocess the DataFrame
def preprocess_df(df):
    # Remove 'INR' prefix from the 'Price' column and convert to float
    df['Price'] = df['Price'].replace({'INR ': ''}, regex=True).astype(float)

    # Replace NaN values in 'Price' with 0.0
    df['Price'].fillna(0.0, inplace=True)

    # Replace NaN values in other columns with appropriate default values
    df.fillna({
        'Route_Name': '',
        'Route_Link': '',
        'Bus_Name': '',
        'Bus_Type': '',
        'Departing_Time': '00:00:00',
        'Duration': '',
        'Reaching_Time': '00:00:00',
        'Star_Rating': 0.0,
        'Seat_Availability': ''
    }, inplace=True)

    return df

# Function to insert data from DataFrame to MySQL
def insert_data_from_df_to_mysql(df):
    try:
        # Establish connection to the MySQL database
        connection = mysql.connector.connect(
            host='127.0.0.1',
            port='3306',
            user='root',
            password='1234',
            database='redbus_1'
        )

        cursor = connection.cursor()
        # Create table if it does not exist
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS bus_details (
            Route_Name TEXT,
            Route_Link TEXT,
            Bus_Name TEXT,
            Bus_Type TEXT,
            Departing_Time TIME,
            Duration TEXT,
            Reaching_Time TIME,
            Star_Rating FLOAT,
            Price DECIMAL(10,2),
            Seat_Availability VARCHAR(255)
        )
        """)

        # Insert data into the table
        for _, detail in df.iterrows():
            cursor.execute("""
            INSERT INTO bus_details (
                Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time,
                Duration, Reaching_Time, Star_Rating, Price, Seat_Availability
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                detail['Route_Name'], detail['Route_Link'], detail['Bus_Name'],
                detail['Bus_Type'], detail['Departing_Time'], detail['Duration'],
                detail['Reaching_Time'], detail['Star_Rating'], detail['Price'],
                detail['Seat_Availability']
            ))

        # Commit the transaction
        connection.commit()
        cursor.close()
        connection.close()

        print("Data inserted successfully into MySQL database.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

# List of CSV files to be inserted into the database
csv_files = [
    r'C:\Users\Administrator\Desktop\redbus\jksrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\apsrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\ctutc_bus_deatails.csv',
    r'C:\Users\Administrator\Desktop\redbus\hrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\ksrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\ktcl_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\sbstc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\tsrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\upsrtc_bus_details.csv',
    r'C:\Users\Administrator\Desktop\redbus\wbtc_bus_details.csv'
]

# Loop through each CSV file, preprocess the data, and insert it into MySQL
for csv_file in csv_files:
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        df = preprocess_df(df)
        insert_data_from_df_to_mysql(df)
    else:
        print(f"File not found: {csv_file}")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)


Data inserted successfully into MySQL database.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)


...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



Data inserted successfully into MySQL database.
File not found: C:\Users\Administrator\Desktop\redbus\tsrtc_bus_details.csv


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)


Data inserted successfully into MySQL database.
Data inserted successfully into MySQL database.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13388\1652545160.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(0.0, inplace=True)
